# T1 Mapping - Radial Look Locker Sequence

T1 mapping using a continuous golden radial Look Locker inversion recovery sequence.

### Imports

In [ ]:
import tempfile
from pathlib import Path

import matplotlib.pyplot as plt
import MRzeroCore as mr0
import numpy as np
import torch
from cmap import Colormap
from einops import rearrange
from mrpro.algorithms.reconstruction import DirectReconstruction
from mrpro.data import CsmData
from mrpro.data import KData
from mrpro.data import SpatialDimension
from mrpro.data.traj_calculators import KTrajectoryIsmrmrd
from mrpro.operators import DictionaryMatchOp
from mrpro.operators.models import TransientSteadyStateWithPreparation
from mrpro.phantoms.coils import birdcage_2d

from mrseq.scripts.t1_radial_look_locker import main as create_seq
from mrseq.utils import combine_ismrmrd_files
from mrseq.utils import sys_defaults

### Settings
We are going to use a numerical phantom with a matrix size of 120 x 120.

In [ ]:
image_matrix_size = [120, 120]
rf_flip_angle = 9

tmp = tempfile.TemporaryDirectory()
fname_mrd = Path(tmp.name) / 't1_radial_look_locker.mrd'

### Create the digital phantom

We use the standard Brainweb phantom from [MRzero](https://github.com/MRsources/MRzero-Core).

In [ ]:
im_dims = SpatialDimension(z=1, y=image_matrix_size[1], x=image_matrix_size[0])
coil_maps = birdcage_2d(6, image_dimensions=im_dims, relative_radius=0.8)

phantom = mr0.util.load_phantom(image_matrix_size)
phantom.coil_sens = rearrange(coil_maps[0, ...], 'coils z y x -> coils x y z')

### Create the T1 radial Look Locker sequence

To create the T1 radial Look Locker sequence, we use the previously imported [t1_radial_look_locker script](../src/mrseq/scripts/t1_radial_look_locker.py).


In [ ]:
sequence, fname_seq = create_seq(
    system=sys_defaults,
    fov_xy=float(phantom.size.numpy()[0]),
    n_readout=image_matrix_size[0],
    n_spokes=600,
    rf_flip_angle=rf_flip_angle,
    test_report=False,
    timing_check=False,
)

### Simulate the sequence
Now, we pass the sequence and the phantom to the MRzero simulation and save the simulated signal as an (ISMR)MRD file.

In [ ]:
mr0_sequence = mr0.Sequence.import_file(str(fname_seq.with_suffix('.seq')))
signal, ktraj_adc = mr0.util.simulate(mr0_sequence, phantom, accuracy=1e-1)
mr0.sig_to_mrd(fname_mrd, signal, sequence)
combine_ismrmrd_files(fname_mrd, Path(f'{fname_seq}_header.h5'))

### Reconstruct the images at different inversion times

We use [MRpro](https://github.com/PTB-MR/MRpro) for the image reconstruction.

In [ ]:
kdata = KData.from_file(str(fname_mrd).replace('.mrd', '_with_traj.mrd'), trajectory=KTrajectoryIsmrmrd())
csm = CsmData.from_kdata_inati(kdata[..., 200:, :], downsampled_size=64, smoothing_width=9)

n_acq_per_image = 32
n_overlap = 16

split_indices = torch.arange(kdata.shape[-2]).unfold(dimension=0, size=n_acq_per_image, step=n_overlap)
kdata_split = kdata[..., split_indices, :]

recon = DirectReconstruction(kdata_split, csm=csm)
idata = recon(kdata_split)

We visualize the coil sensitivity maps which we used for the iterative SENSE reconstruction.

In [ ]:
fig, ax = plt.subplots(1, csm.shape[1], figsize=(9 * csm.shape[0], 9))
for i in range(csm.shape[1]):
    ax[i].imshow(csm.data[0, i, 0, :, :].abs(), cmap='gray')

We can now plot the images at different time points along the acquisition, i.e. at different inversion times.

In [ ]:
idat = idata.data.abs().numpy().squeeze()
fig, ax = plt.subplots(6, idat.shape[0] // 6, figsize=(4 * idata.shape[0] // 6, 6 * 4))
ax = ax.flatten()
for i in range(idat.shape[0]):
    ax[i].imshow(idat[i, :, :], cmap='gray')
    ax[i].set_xticks([])
    ax[i].set_yticks([])

### Estimate the T1 maps
We use a dictionary matching approach to estimate the T1 maps. Afterward, we compare them to the input and ensure they match.

In [ ]:
sampling_time = idata.header.acquisition_time_stamp - kdata.header.acq_info.acquisition_time_stamp[0, 0, 0, 0, 0]
dictionary = DictionaryMatchOp(
    TransientSteadyStateWithPreparation(
        sampling_time=sampling_time.to(torch.float32),
        repetition_time=idata.header.tr,
        m0_scaling_preparation=-1,
        delay_after_preparation=idata.header.ti,
    ),
    index_of_scaling_parameter=0,
)
dictionary.append(
    torch.tensor(1.0),
    torch.linspace(0.1, 5.0, 1000)[None, :, None],
    torch.deg2rad(torch.linspace(rf_flip_angle * 0.8, rf_flip_angle * 1.2, 100))[None, None, :],
)
m0_match, t1_match, fa_match = dictionary(idata.data)

t1_input = np.roll(rearrange(phantom.T1.numpy().squeeze()[::-1, ::-1], 'x y -> y x'), shift=(1, 1), axis=(0, 1))
fa_input = (
    np.abs(np.roll(rearrange(phantom.B1.numpy().squeeze()[::-1, ::-1], 'x y -> y x'), shift=(1, 1), axis=(0, 1))) * 9
)
obj_mask = np.zeros_like(t1_input)
obj_mask[t1_input > 0] = 1
fa_input = fa_input * obj_mask
t1_measured = t1_match.numpy().squeeze() * obj_mask
fa_measured = np.rad2deg(fa_match.numpy().squeeze() * obj_mask)

fig, ax = plt.subplots(2, 3, figsize=(15, 6))
for cax in ax.flatten():
    cax.set_xticks([])
    cax.set_yticks([])

im = ax[0, 0].imshow(t1_input, vmin=0, vmax=1.8, cmap=Colormap('lipari').to_mpl())
fig.colorbar(im, ax=ax[0, 0], label='Input T1 (s)')

im = ax[0, 1].imshow(t1_measured, vmin=0, vmax=1.8, cmap=Colormap('lipari').to_mpl())
fig.colorbar(im, ax=ax[0, 1], label='Measured T1 (s)')

im = ax[0, 2].imshow(t1_measured - t1_input, vmin=-1.8, vmax=1.8, cmap='bwr')
fig.colorbar(im, ax=ax[0, 2], label='Difference T1 (s)')

im = ax[1, 0].imshow(fa_input, vmin=0, vmax=12, cmap='magma')
fig.colorbar(im, ax=ax[1, 0], label='Input Flip Angle (°)')

im = ax[1, 1].imshow(fa_measured, vmin=0, vmax=12, cmap='magma')
fig.colorbar(im, ax=ax[1, 1], label='Measured Flip Angle (°)')

im = ax[1, 2].imshow(fa_measured - fa_input, vmin=-12, vmax=12, cmap='bwr')
fig.colorbar(im, ax=ax[1, 2], label='Difference Flip Angle (°)')

relative_error = np.sum(np.abs(t1_input - t1_measured)) / np.sum(np.abs(t1_input))
print(f'Relative error of T1 {relative_error}')
assert relative_error < 0.08

relative_error = np.sum(np.abs(fa_input - fa_measured)) / np.sum(np.abs(fa_input))
print(f'Relative error of Flip Angle {relative_error}')
assert relative_error < 0.06